In [126]:
import pandas as pd

In [127]:
ref = pd.read_excel(r'C:\Users\sgasmi\Desktop\Dev\UO\Référentiel UO.xlsx')
ref = ref.loc[ref["Code type d'unité de tr."]!="00"] # Suppression du niveau 0
ref.reset_index(drop=True, inplace=True)

In [128]:
col = ["1","2","3","4","5","6","7","8"]
result = pd.DataFrame(columns=col)

In [129]:
print(ref.loc[ref["Code de l'unité de travail"]=="ROOT"])

Empty DataFrame
Columns: [Code de l'unité de travail, Nom de l'unité de travail, Code type d'unité de tr., Objectifs, Date de début, Date de fin, Description, Code unité de tr. mère, Code type d'unité de tr..1, Date de début rattachement, Date de fin rattachement, Code externe UT, Code de l'unité de travail.1, Matricule Manager, Nom Manager, Date de début Manager, Prénom Manager, Matricule RH, Nom RH, Date de début RH, Prénom RH]
Index: []

[0 rows x 21 columns]


In [130]:
for i, val in enumerate(ref.iterrows()):
    col_cible = ref.at[i, "Code type d'unité de tr."][1]
    # Alimentation de la première colonne
    result.at[i, col_cible] = ref.at[i,"Nom de l'unité de travail"]
    #Alimentation des autres colonnes
    cmere = ""

    while int(col_cible)>1: 
        if cmere== "":
            cmere=ref.at[i,"Code unité de tr. mère"]
        try:
            nuo = ref.loc[ref["Code de l'unité de travail"]==cmere,"Nom de l'unité de travail"].iloc[0]
        except:
            print(ref.at[i,"Nom de l'unité de travail"])
        col_cible= str(int(col_cible)-1)
        result.at[i, col_cible] = nuo
        cmere = ref.loc[ref["Code de l'unité de travail"]==cmere,"Code unité de tr. mère"].iloc[0]
        
     
        #result.at[i, col_cible-1] = ref.loc[ref["Nom de l'unité de travail"]==ref.at[i,"Code unité de tr. mère"]]
result = result.merge(ref[["Code de l'unité de travail","Nom de l'unité de travail","Code type d'unité de tr.", "Description"]], left_index=True, right_index=True)

In [131]:
# Réorganisation des colonnes

col = result.columns
l1 = list(col[-3:])
l2 = list(list(col[:-3]))
col = l1 + l2

result = result[col]
print(result.columns)
result.columns=["Nom de l'unité de travail","Code type d'unité de tr.", "Description",
       "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8", "Code de l'unité de travail", ]

result = result[["Code de l'unité de travail","Nom de l'unité de travail", "Code type d'unité de tr.", "Description",
       "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8"]]
#print(list(col[-2:] + list(col[:-2]))

Index(['Nom de l'unité de travail', 'Code type d'unité de tr.', 'Description',
       '1', '2', '3', '4', '5', '6', '7', '8', 'Code de l'unité de travail'],
      dtype='object')


### Ajout de l'UO mère

In [132]:
mere = ref[["Code de l'unité de travail", "Code unité de tr. mère"]]
result = result.merge(mere, on="Code de l'unité de travail", how="left")
mere = ref[["Code de l'unité de travail", "Nom de l'unité de travail"]]
result = result.merge(mere, left_on="Code unité de tr. mère", right_on="Code de l'unité de travail", how="left")
result = result.drop(columns=["Code unité de tr. mère", "Code de l'unité de travail_y"], axis=1)
result = result.rename(columns={"Nom de l'unité de travail_y": "UO mère", "Code de l'unité de travail_x":"Code de l'unité de travail", "Nom de l'unité de travail_x": "Nom de l'unité de travail"})
result.loc[result["UO mère"].isna()==True, "UO mère"] = "ROOT"

In [133]:
result.to_excel(r'C:\Users\sgasmi\Desktop\Export Python\UO_Org.xlsx', index=False)